In [29]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

import seaborn as sns
import os
from keras.applications import VGG19
from os import listdir, makedirs
from os.path import join, exists, expanduser
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from numpy import newaxis
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dense,Activation,Dropout,Flatten,BatchNormalization
import matplotlib.pyplot as plt
from glob import glob
from keras.applications import VGG19
from skimage.feature import local_binary_pattern
from tensorflow.keras.optimizers import Adam,SGD
import numpy as np
from tensorflow.keras.regularizers import l2
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from PIL import Image

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
IMAGE_SIZE=331
BATCH_SIZE=64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=.1/255)
train_generator = datagen.flow_from_directory("/content/drive/MyDrive/Car_Brand_Logos/Train",
target_size=(IMAGE_SIZE,IMAGE_SIZE),
class_mode="categorical",
color_mode="rgb",
batch_size=BATCH_SIZE)

datagen = tf.keras.preprocessing.image.ImageDataGenerator()
test_generator = datagen.flow_from_directory("/content/drive/MyDrive/Car_Brand_Logos/Test",
target_size=(IMAGE_SIZE,IMAGE_SIZE),
class_mode="categorical",
color_mode="rgb",
batch_size=BATCH_SIZE)



Found 2513 images belonging to 8 classes.
Found 408 images belonging to 8 classes.


In [35]:
vgg = VGG19(include_top= False, weights = "imagenet", input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))
vgg_layer_list = vgg.layers

model = Sequential ()

for layer in vgg_layer_list:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False



model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(8))
model.add(Activation('softmax'))
model.summary()

opt = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)                │ (None, 331, 331, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 331, 331, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 165, 165, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 165, 165, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 165, 165, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 82, 82, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 82, 82, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 41, 41, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 41, 41, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 20, 20, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 10, 10, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 72,983,112 (278.41 MB)

 Trainable params: 52,958,728 (202.02 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)                │ (None, 331, 331, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 331, 331, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 165, 165, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 165, 165, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 165, 165, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 82, 82, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 82, 82, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 82, 82, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 41, 41, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 41, 41, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 41, 41, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 20, 20, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 20, 20, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 10, 10, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 72,983,112 (278.41 MB)

 Trainable params: 52,958,728 (202.02 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [37]:
epochs=10
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint ('model.{epoch:02d}.keras',
                              monitor='val_loss',
                              verbose=0,
                              save_best_only=True,
                              mode='auto')
history = model. fit(train_generator,
                     steps_per_epoch=len(train_generator),
                     epochs=epochs,
                     validation_data=test_generator,
                     validation_steps=len(test_generator),
                     callbacks=[checkpoint])

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.5179 - loss: 1.2971 - val_accuracy: 0.2647 - val_loss: 186.5610
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5793 - loss: 1.1940 - val_accuracy: 0.2892 - val_loss: 157.8284
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.5734 - loss: 1.1824 - val_accuracy: 0.2721 - val_loss: 172.8728
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.5901 - loss: 1.1258 - val_accuracy: 0.2696 - val_loss: 187.6634
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6105 - loss: 1.0988 - val_accuracy: 0.2574 - val_loss: 204.3513
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.6409 - loss: 1.0175 - val_accuracy: 0.2598 - val_loss: 231.5227
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.6364 - loss: 0.9920 - val_accuracy: 0.2647 - val_loss: 220.6719
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.6277 - loss: 1.0558 - val_accuracy: 0.23

In [38]:
class_names=train_generator.class_indices

labels={}
for key,value in class_names.items():
    labels[value]=key

In [39]:
labels

{0: 'hyundai',
 1: 'lexus',
 2: 'mazda',
 3: 'mercedes',
 4: 'opel',
 5: 'skoda',
 6: 'toyota',
 7: 'volkswagen'}

In [49]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np


loaded_model = load_model("/content/model.01.keras")


def preprocess_image(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array


image_path = '/content/drive/MyDrive/2020-skoda-rapid-6.jpg'

preprocessed_image = preprocess_image(image_path)


predictions = loaded_model.predict(preprocessed_image)

predicted_class = np.argmax(predictions, axis=1)


print('Predicted class:', predicted_class)

predicted_class_name = labels[predicted_class[0]]

print('Predicted class:', predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
Predicted class: [5]
Predicted class: skoda
